In [2]:
import numpy as np
from keras import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications import VGG19,Xception
from tensorflow.keras.layers import Input,Flatten,Dense,BatchNormalization,Activation,Dropout,GlobalAveragePooling2D,MaxPooling2D,RandomFlip,RandomZoom,RandomRotation
from keras.datasets import cifar10

In [4]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [5]:
base_model = Xception(include_top=False, weights='imagenet', input_shape=(224,224,3), classes=y_train.shape[1])
base_model.trainable = False

In [8]:
import tensorflow as tf
inputs = tf.keras.Input(shape=(32, 32, 3))
x = tf.keras.layers.Lambda(lambda image: tf.image.resize(image, (224,224)))(inputs)
x = tf.keras.applications.xception.preprocess_input(x)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(10, activation=('softmax'))(x)
model = tf.keras.Model(inputs, outputs)

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
from keras.callbacks import EarlyStopping
epochs = 10
batch_size = 32
early_stopping = EarlyStopping(
    monitor='accuracy',
    patience = 1,
    restore_best_weights = True
)
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping])

Epoch 1/10
1563/1563 [==============================] - 2893s 2s/step - loss: 0.4818 - accuracy: 0.8405
Epoch 2/10
1563/1563 [==============================] - 2877s 2s/step - loss: 0.3841 - accuracy: 0.8687
Epoch 3/10
1563/1563 [==============================] - 2872s 2s/step - loss: 0.3613 - accuracy: 0.8760
Epoch 4/10
1563/1563 [==============================] - 2892s 2s/step - loss: 0.3516 - accuracy: 0.8802
Epoch 5/10
1563/1563 [==============================] - 2879s 2s/step - loss: 0.3449 - accuracy: 0.8806
Epoch 6/10
1563/1563 [==============================] - 2876s 2s/step - loss: 0.3413 - accuracy: 0.8821
Epoch 7/10
1563/1563 [==============================] - 2876s 2s/step - loss: 0.3368 - accuracy: 0.8843
Epoch 8/10
1563/1563 [==============================] - 2872s 2s/step - loss: 0.3363 - accuracy: 0.8844
Epoch 9/10
1563/1563 [==============================] - 2881s 2s/step - loss: 0.3344 - accuracy: 0.8860
Epoch 10/10
1563/1563 [==============================] - 2881s 2

In [11]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

313/313 - 576s - loss: 0.3823 - accuracy: 0.8724 - 576s/epoch - 2s/step

Test accuracy: 0.8723999857902527


In [13]:
model.save('xception_cifar.h5')